In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.signal import find_peaks

def moving_average(data, window_size):
    weights = np.repeat(1.0, window_size) / window_size
    return np.convolve(data, weights, 'valid')






# Extraction of Spatio-temporal Parameters from MOCAP

In [ ]:
#Average step length\ average stride length\stride time\stride speed
def mocap_gait_parameter_1(data):
    #Calculate the Y-axis difference between points 19 and 20 of the ankle
    diff = [(data['Y19'][i] - data['Y20'][i]) for i in range(len(data))]
    diff = np.array(diff)
    # Smoothing and Peak Detection
    window_size = 3
    smoothed_y = moving_average(diff, window_size)
    peak, _ = find_peaks(abs(smoothed_y))   
    
    peak_num = len(peak)
    if peak_num == 0:
       aver_step_length = abs((data['Y19'][len(data)-1]- data['Y19'][0]))/2
       aver_stride_length = abs((data['Y19'][len(data)-1]- data['Y19'][0]))
       aver_stride_time =data['Time'][len(data)-1] - data['Time'][0]
       aver_stride_speed = aver_stride_length/aver_stride_time

       return aver_step_length, aver_stride_length, aver_stride_time,aver_stride_speed
    elif  peak_num == 1:
        aver_step_length=abs(diff[peak[0]])
        aver_stride_length = aver_step_length * 2
        aver_stride_time =data['Time'][len(data)-1] - data['Time'][0]
        aver_stride_speed = aver_stride_length/aver_stride_time
        return aver_step_length, aver_stride_length, aver_stride_time,aver_stride_speed
    else:
        # Find the indices of elements to be removed and store them in the 'to_remove' list
        to_remove = []
        t=diff[peak[0]]
        p=0
        for num in range(1,len(peak)):
            if diff[peak[num]] * diff[peak[num - 1]] > 0:
                if abs(diff[peak[num]]) < t:
                    to_remove.append(num)
                else:
                    to_remove.append(p)
                    t=abs(diff[peak[num]])
                    p = num 
            else:
                t=abs(diff[peak[num]])
                p=num
        
        
        new_peak = np.delete(peak, to_remove)
        
        if  len(new_peak) == 1:
            aver_step_length=abs(diff[new_peak[0]])
            aver_stride_length = aver_step_length *2
            aver_stride_time =data['Time'][len(data)-1] - data['Time'][0]
            aver_stride_speed = aver_stride_length/aver_stride_time
            return aver_step_length, aver_stride_length,aver_stride_time, aver_stride_speed
        else:
            
            step_length = 0
            for _id in range(len(new_peak)):
                step_length += abs(diff[new_peak[_id]])
            aver_step_length = step_length / len(new_peak)
           
            stride_length = 0
            for _id2 in range(len(new_peak) - 1):
                stride_length += abs(diff[new_peak[_id2 + 1]] - diff[new_peak[_id2]])
            aver_stride_length = stride_length / (len(new_peak) - 1)
            speed = 0
            for _id3 in  range(len(new_peak) - 1):
                stride_length = abs(diff[new_peak[_id3 + 1]] - diff[new_peak[_id3]])
                stride_time = data['Time'][new_peak[_id3 + 1]]-data['Time'][new_peak[_id3]]
                speed += stride_length / stride_time
            aver_stride_speed = speed / (len(new_peak)-1)
            stride_time = 0
            for _id4 in  range(len(new_peak) - 1):
                stride_time += data['Time'][new_peak[_id4 + 1]]-data['Time'][new_peak[_id4]]
            aver_stride_time = stride_time / (len(new_peak)-1)
            return aver_step_length, aver_stride_length, aver_stride_time, aver_stride_speed



#left step length/right step length/left stride length/right stride length
def mocap_gait_parameter_2(data):
    
    diff = [(data['Y19'][i] - data['Y20'][i]) for i in range(len(data))]
    diff = np.array(diff)
    
    window_size = 3
    smoothed_y = moving_average(diff, window_size)
    peak, _ = find_peaks(abs(smoothed_y))
 
    left_step = 0
    left_i=0
    right_step = 0
    right_i = 0
    left_stride = 0
    right_stride = 0
    if len(peak) == 0:
        aver_left_step = 0
        aver_right_step = 0
        aver_left_stride = 0
        aver_right_stride = 0
    elif len(peak)==1:
        if diff[peak[0]]>0:
             aver_left_step = diff[peak[0]]
             aver_right_step = 0
             aver_left_stride = 0 
             aver_right_stride = 0 
        else:
             aver_left_step = 0
             aver_right_step = abs(diff[peak[0]])
             aver_left_stride = 0 
             aver_right_stride = 0
    else:
        
        to_remove = []
        t=diff[peak[0]]
        p=0
        for num in range(1,len(peak)):
            if diff[peak[num]] * diff[peak[num - 1]] > 0:
                if abs(diff[peak[num]]) < t:
                    to_remove.append(num)
                else:
                    to_remove.append(p)
                    t=abs(diff[peak[num]])
                    p = num 
            else:
                t=abs(diff[peak[num]])
                p=num
        
       
        new_peak = np.delete(peak, to_remove)
        
        if len(new_peak)==1:
            if diff[new_peak[0]]>0:
                 aver_left_step = diff[new_peak[0]]
                 aver_right_step = 0
                 aver_left_stride = 0 
                 aver_right_stride = 0  
            else:
                 aver_left_step = 0
                 aver_right_step = abs(diff[new_peak[0]])
                 aver_left_stride = 0
                 aver_right_stride = 0
        elif len(new_peak)==2:
            if diff[new_peak[0]]>0:
                aver_left_step = abs(diff[new_peak[0]])
                aver_right_step = abs(diff[new_peak[1]])
                aver_left_stride = 0
                aver_right_stride = abs(diff[new_peak[0]])+abs(diff[new_peak[1]])
            else:
                aver_left_step = abs(diff[new_peak[1]])
                aver_right_step = abs(diff[new_peak[0]])
                aver_left_stride = abs(diff[new_peak[0]])+abs(diff[new_peak[1]])
                aver_right_stride = 0 
        else:
            for i in range(len(new_peak)-1):
               if diff[new_peak[i]]>0:
                    left_step += diff[new_peak[i]]
                    left_i+=1
                    right_stride += abs(diff[new_peak[i]])+abs(diff[new_peak[i+1]])
               else:
                    right_step +=abs(diff[new_peak[i]])
                    right_i+=1
                    left_stride += abs(diff[new_peak[i]])+abs(diff[new_peak[i+1]])
            if diff[new_peak[-1]]>0:
                left_step = left_step+diff[new_peak[-1]]
                left_i+=1
                aver_left_step = left_step / left_i 
                aver_right_step = right_step / right_i
                aver_left_stride = left_stride / right_i
                aver_right_stride = right_stride/(left_i-1)
            else:
                right_step = right_step+abs(diff[new_peak[-1]])
                right_i+=1
                aver_left_step = left_step / left_i 
                aver_right_step = right_step / right_i
                aver_left_stride = left_stride / (right_i-1)
                aver_right_stride = right_stride/left_i
        
    return aver_left_step, aver_right_step,aver_left_stride,aver_right_stride


#step width
def mocap_step_width(data):
    w = abs(data['X19']-data['X20'])
    aver_stride_width = abs(sum(w)/len(w))
    return aver_stride_width


In [ ]:

def mocap_gait_feature(file_path):
    
    mocap_gait_parameter = pd.DataFrame(columns=['device','sample', 'times', 'process', 'task', 'step_length','stride_length','left_step_length','right_step_length','left_stride_length','right_stride_length','stride_speed','stride_time','step_width'])
    t = 0
   
    contents = os.listdir(file_path)
    for item in contents[:]:
        item_path = os.path.join(file_path, item) 
        print('folder:' + item_path)
        content_item = os.listdir(item_path)

        # go/back/turn folders
        for process in content_item[:]:
            process_path = os.path.join(item_path, process)
            print('folder:' + process_path)
            item_contents = os.listdir(process_path)

            # 'task' folders
            for task in item_contents:
                task_path = os.path.join(process_path, task)
                print("folder:" + task_path)
                task_contents = os.listdir(task_path)
                # 23 files
                for num in task_contents:
                    num_path = os.path.join(task_path, num)
                    print("folder:" + num_path)
                    
                    num1 = pd.read_csv(num_path)
                    if len(num1)<1:
                        continue
                        
                    #Calculate the step length and stride length and stride time and stride speed
                    aver_step_length,aver_stride_length,aver_stride_time,aver_stride_speed= mocap_gait_parameter_1(num1)
                    
                    
                    #Calculate the left step length/right step length/left stride length/right stride length
                    aver_left_step, aver_right_step,aver_left_stride,aver_right_stride = mocap_gait_parameter_2(num1)
                
                    #Calculate the step width
                    aver_step_width = mocap_step_width(num1)
                    
                    
            

                    mocap_gait_parameter.loc[t] = ['MOCAP',num[:8], item, process, task, aver_step_length, aver_stride_length, aver_left_step, aver_right_step, aver_left_stride, aver_right_stride,aver_stride_speed,aver_stride_time,aver_step_width]
                    t = t + 1

    mocap_gait_parameter.to_csv("\Data_new\\Parameters\\MOCAP_5m\\spatio-temporal_parameters.csv",index=False)





In [ ]:
mocap_gait_feature("\Data_new\\Processed\\MOCAP_5m")

# Extraction of Spatio-temporal Parameters from Kinect

In [ ]:
#Average step length\ average stride length\stride time\stride speed
def kinect_gait_parameter_1(data):
    #Calculate the Z-axis difference between points 14 and 18 of the ankle
    diff = [(data['Y14'][i] - data['Y18'][i]) for i in range(len(data))]
    diff = np.array(diff)

    window_size = 3
    smoothed_y = moving_average(diff, window_size)
    peak, _ = find_peaks(abs(smoothed_y))   

    peak_num = len(peak)
    if peak_num == 0:
       aver_stride_length = abs((data['Y14'][len(data)-1]- data['Y14'][0]))
       aver_step_length = abs((data['Y14'][len(data)-1]- data['Y14'][0]))/2
       aver_stride_time =data['Timestamp'][len(data)-1] - data['Timestamp'][0]
       aver_stride_speed = aver_step_length/aver_stride_time
       
       return aver_step_length, aver_stride_length, aver_stride_speed,aver_stride_time
    elif  peak_num == 1:
        aver_step_length=abs(diff[peak[0]])
        aver_stride_length = aver_step_length *2
        aver_stride_time =data['Timestamp'][len(data)-1] - data['Timestamp'][0]
        aver_stride_speed = aver_stride_length/aver_stride_time
        return aver_step_length, aver_stride_length,aver_stride_time, aver_stride_speed
    else:
  
        to_remove = []
        t=diff[peak[0]]
        p=0
        for num in range(1,len(peak)):
            if diff[peak[num]] * diff[peak[num - 1]] > 0:
                if abs(diff[peak[num]]) < t:
                    to_remove.append(num)
                else:
                    to_remove.append(p)
                    t=abs(diff[peak[num]])
                    p = num 
            else:
                t=abs(diff[peak[num]])
                p=num
        
  
        new_peak = np.delete(peak, to_remove)
        
        if  len(new_peak) == 1:
            aver_step_length=abs(diff[new_peak[0]])
            aver_stride_length = aver_step_length *2
            aver_stride_time =data['Timestamp'][len(data)-1] - data['Timestamp'][0]
            aver_stride_speed = aver_stride_length/aver_stride_time

            return aver_step_length, aver_stride_length, aver_stride_time,aver_stride_speed
        else:
            
            step_length = 0
            for _id in range(len(new_peak)):
                step_length += abs(diff[new_peak[_id]])
            aver_step_length = step_length / len(new_peak)
            
            stride_length = 0
            for _id2 in range(len(new_peak) - 1):
                stride_length += abs(diff[new_peak[_id2 + 1]] - diff[new_peak[_id2]])
            aver_stride_length = stride_length / (len(new_peak) - 1)
            stride_speed = 0
            for _id3 in  range(len(new_peak) - 1):
                stride_length = abs(diff[new_peak[_id3 + 1]] - diff[new_peak[_id3]])
                stride_time = data['Timestamp'][new_peak[_id3 + 1]]-data['Timestamp'][new_peak[_id3]]
                stride_speed += stride_length / stride_time
            aver_stride_speed = stride_speed / (len(new_peak)-1)
            stride_time = 0
            for _id4 in  range(len(new_peak) - 1):
                stride_time += data['Timestamp'][new_peak[_id4 + 1]]-data['Timestamp'][new_peak[_id4]]
            aver_stride_time = stride_time / (len(new_peak)-1)
            return aver_step_length, aver_stride_length, aver_stride_time, aver_stride_speed


#left step length/right step length/left stride length/right stride length
def kinect_gait_parameter_2(data):
  
    diff = [(data['Y14'][i] - data['Y18'][i]) for i in range(len(data))]
    diff = np.array(diff)
    
    window_size = 3
    smoothed_y = moving_average(diff, window_size)
    peak, _ = find_peaks(abs(smoothed_y))
 
    left_step = 0
    left_i=0
    right_step = 0
    right_i = 0
    left_stride = 0
    right_stride = 0
    if len(peak) == 0:
        aver_left_step = 0
        aver_right_step = 0
        aver_left_stride = 0
        aver_right_stride = 0 
    elif len(peak)==1:
        if diff[peak[0]]>0:
             aver_left_step = diff[peak[0]]
             aver_right_step = 0
             aver_left_stride = 0
             aver_right_stride = 0 
        else:
             aver_left_step = 0
             aver_right_step = abs(diff[peak[0]])
             aver_left_stride = 0
             aver_right_stride = 0
    else:
      
        to_remove = []
        t=diff[peak[0]]
        p=0
        for num in range(1,len(peak)):
            if diff[peak[num]] * diff[peak[num - 1]] > 0:
                if abs(diff[peak[num]]) < t:
                    to_remove.append(num)
                else:
                    to_remove.append(p)
                    t=abs(diff[peak[num]])
                    p = num 
            else:
                t=abs(diff[peak[num]])
                p=num
        
    
        new_peak = np.delete(peak, to_remove)
        
        if len(new_peak)==1:
            if diff[new_peak[0]]>0:
                 aver_left_step = diff[new_peak[0]]
                 aver_right_step = 0
                 aver_left_stride = 0
                 aver_right_stride = 0 
            else:
                 aver_left_step = 0
                 aver_right_step = abs(diff[new_peak[0]])
                 aver_left_stride = 0
                 aver_right_stride = 0
        elif len(new_peak)==2:
            if diff[new_peak[0]]>0:
                aver_left_step = abs(diff[new_peak[0]])
                aver_right_step = abs(diff[new_peak[1]])
                aver_left_stride = 0
                aver_right_stride = abs(diff[new_peak[0]])+abs(diff[new_peak[1]]) 
            else:
                aver_left_step = abs(diff[new_peak[1]])
                aver_right_step = abs(diff[new_peak[0]])
                aver_left_stride = abs(diff[new_peak[0]])+abs(diff[new_peak[1]])
                aver_right_stride = 0
        else:
            for i in range(len(new_peak)-1):
               if diff[new_peak[i]]>0:
                    left_step += diff[new_peak[i]]
                    left_i+=1
                    right_stride += abs(diff[new_peak[i]])+abs(diff[new_peak[i+1]])
               else:
                    right_step +=abs(diff[new_peak[i]])
                    right_i+=1
                    left_stride += abs(diff[new_peak[i]])+abs(diff[new_peak[i+1]])
            if diff[new_peak[-1]]>0:
                left_step = left_step+diff[new_peak[-1]]
                left_i+=1
                aver_left_step = left_step / left_i 
                aver_right_step = right_step / right_i
                aver_left_stride = left_stride / right_i
                aver_right_stride = right_stride/(left_i-1)
            else:
                right_step = right_step+abs(diff[new_peak[-1]])
                right_i+=1
                aver_left_step = left_step / left_i 
                aver_right_step = right_step / right_i
                aver_left_stride = left_stride / (right_i-1)
                aver_right_stride = right_stride/left_i
        
    return aver_left_step, aver_right_step,aver_left_stride,aver_right_stride
        
#step width
def kinect_step_width(data):
    w = abs(data['X14']-data['X18'])
    aver_stride_width = abs(sum(w)/len(w))
    return aver_stride_width




In [ ]:
def kinect_gait_feature(file_path):
    
    kinect_gait_parameter = pd.DataFrame(columns=['device','sample', 'times', 'process', 'task', 'step_length','stride_length','left_step_length','right_step_length','left_stride_length','right_stride_length','stride_speed','stride_time','step_width'])
    t = 0

    contents = os.listdir(file_path)
    for item in contents[:]:
        item_path = os.path.join(file_path, item)  # 样本文件夹所在位置
        print('folder:' + item_path)
        content_item = os.listdir(item_path)

        
        for process in content_item[:]:
            process_path = os.path.join(item_path, process)
            print('folder:' + process_path)
            process_contents = os.listdir(process_path)

          
            for task in process_contents:
                task_path = os.path.join(process_path, task)
                print("folder:" + task_path)
                task_contents = os.listdir(task_path)
                
                for num in task_contents:
                    num_path = os.path.join(task_path, num)
                    print("folder:" + num_path)
                 
                    num1 = pd.read_csv(num_path)
                    if len(num1)<1:
                        continue
                  
                    aver_step_length,aver_stride_length,aver_stride_time,aver_stride_speed= kinect_gait_parameter_1(num1)
                    
                    
                   
                    aver_left_step, aver_right_step,aver_left_stride,aver_right_stride = kinect_gait_parameter_2(num1)
                
              
                    aver_step_width = kinect_step_width(num1)
                    
                    
        

                    kinect_gait_parameter.loc[t] = ['Kinect',num[:8], item, process, task, aver_step_length, aver_stride_length, aver_left_step, aver_right_step, aver_left_stride, aver_right_stride,aver_stride_speed,aver_stride_time,aver_step_width]
                    t=t+1

    kinect_gait_parameter.to_csv("\Data_new\\Parameters\\Kinect_2m\\spatio-temporal_parameters.csv",index=False)


In [ ]:
kinect_gait_feature("\Data_new\\Processed\\Kinect_2m")